# SmartPrix Laptop Dataset Analysis

## Market and Value Analysis

Hey there! 

Welcome to our `Market and Value Analysis` notebook. We're diving into the business side of the Indian laptop market using our cleaned SmartPrix Laptop dataset. 

We'll explore **pricing trends, brand comparisons, market share and value propositions** aiming to reveal insights that can guide purchasing decisions and business strategies. All prices mentioned in the notebook, analyses, outputs and summaries are in Indian Rupees (INR).

We'll also look at how different features impact pricing and which brands dominate in various market segments. Additionally, we'll explore the **adoption of newer technologies across different price segments and brands**. 

This should give us some solid insights into consumer preferences and market dynamics. 

Let's get started and see some of the interesting patterns we can uncover!

### Connecting to the Database
The first line of code connects us to the `smartprix_laptop_db` database where the `laptops` table resides. 
We will also connect to the `laptop_schema` schema, allowing us to start running the queries and exploring the dataset interactively.

In [4]:
import sqlalchemy
%load_ext sql
%sql postgresql:///smartprix_laptop_db
%sql SET search_path TO laptop_schema;

### Q1: Average price of laptops for each brand
What is the average price of laptops for each brand?

In [7]:
%%sql
SELECT brand, 
    COUNT(*) AS laptop_count, 
    ROUND(AVG(price), 2) AS avg_price
FROM laptops
GROUP BY brand
ORDER BY avg_price DESC;

31 rows affected.

brand,laptop_count,avg_price
Razer,1,199990.00
Apple,17,187862.00
Samsung,21,134224.24
MSI,107,123260.88
LG,5,117097.20
Dell,105,106420.15
Gigabyte,6,96991.67
Huawei,6,96830.83
Colorful,1,89999.00
Microsoft,2,86794.50


The laptop market shows a wide range of prices across 31 brands, with **Razer, Apple, and Samsung** leading the **high-end segment** with average prices *above 130,000*. 

**Mid-range brands** like **Dell, HP, Asus, and Lenovo** offer a large variety of models with average prices *between 70,000 and 110,000*, dominating in terms of laptop count. 

**Budget-friendly options** are available from brands like **Infinix, Chuwi, and Avita**, with average prices *below 40,000*, catering to cost-conscious consumers. 

The market demonstrates significant diversity, with average prices ranging from 8,000 (iBall) to 199,990 (Razer), ensuring options for various consumer needs and budgets.

Some brands have a large number of models (Lenovo: 202, HP: 195, Asus: 178), while others have very few (Razer, Colorful, Realme: 1 each).

### Q2: Correlation between price and spec score
What is the correlation between price and spec score?

In [36]:
%%sql
SELECT ROUND(CORR(price, spec_score)::NUMERIC, 2) 
        AS correlation
FROM laptops;

1 rows affected.

correlation
0.72


The analysis indicates a **strong positive correlation** of 0.72 between the price and spec score of laptops. This shows that higher priced laptops tend to have higher specification scores.

### Q3: Price difference between NVIDIA and other graphics cards
Is there a significant price difference between laptops with NVIDIA graphics cards and those with other graphics card?

In [9]:
%%sql
SELECT 
    CASE 
        WHEN graphics_card ILIKE '%Nvidia%' THEN 'NVIDIA Graphics'
        ELSE 'Other Graphics'
    END AS graphics_type,
    COUNT(*) AS laptop_count,
    ROUND(AVG(price), 2) AS average_price
FROM laptops
GROUP BY 
    CASE 
        WHEN graphics_card ILIKE '%Nvidia%' THEN 'NVIDIA Graphics'
        ELSE 'Other Graphics'
    END;

2 rows affected.

graphics_type,laptop_count,average_price
Other Graphics,628,63095.67
NVIDIA Graphics,370,121690.55


Based on the output, laptops with **NVIDIA graphics cards** have a **higher average price** compared to those with other graphic cards. The difference is nearly double the price, suggesting that NVIDIA graphics are associated with higher-end and more expensive laptop models.

### Q4: Most common OS in laptops priced over 100,000
Which operating system is most common among laptops priced over 100,000?

In [10]:
%%sql
SELECT os, COUNT(*) AS laptop_count
FROM laptops
WHERE price > 100000
GROUP BY os
ORDER BY laptop_count DESC;

5 rows affected.

os,laptop_count
Windows 11,213
Mac,15
Windows 10,2
Utility: Everyday Use,1
Windows,1


### Q5: Relationship between RAM capacity and price
What is the relationship between RAM capacity and price?

In [11]:
%%sql
WITH ram_extracted AS (
SELECT price,
CASE WHEN internal_memory LIKE '%TB%' THEN 
    CAST(SUBSTRING(UPPER(internal_memory) FROM '[0-9]+(?=\s*TB)') 
         AS INTEGER) * 1024
    WHEN internal_memory LIKE '%GB%' THEN 
    CAST(SUBSTRING(UPPER(internal_memory) FROM '[0-9]+(?=\s*GB)') 
         AS INTEGER) 
    ELSE 0
        END AS ram_gb
    FROM laptops
),
ram_categories AS (
SELECT CASE 
    WHEN ram_gb <= 128 THEN '0-128 GB'
    WHEN ram_gb <= 512 THEN '129-512 GB' 
    ELSE '513+ GB' 
    END AS ram_category,
price
    FROM ram_extracted
)
SELECT 
    ram_category,
    COUNT(*) AS laptop_count,
    ROUND(AVG(price), 2) AS avg_price,
    MIN(price) AS min_price,
    MAX(price) AS max_price
FROM ram_categories
GROUP BY ram_category
ORDER BY 
CASE WHEN ram_category = '0-128 GB' THEN 1 
     WHEN ram_category = '129-512 GB' THEN 2 
     WHEN ram_category = '513+ GB' THEN 3
END;

3 rows affected.

ram_category,laptop_count,avg_price,min_price,max_price
0-128 GB,23,23709.13,8000,150000
129-512 GB,709,62169.54,15990,231746
513+ GB,266,150473.93,34980,599990


The data shows a **clear correlation** between `RAM capacity` and `laptop prices`. Higher RAM capacities are seen in higher average prices. Laptops in the **highest RAM category (513+ GB)** have an average price **more than six times** than laptops in the **lowest RAM category (0 - 128 GB)**. This clearly shows that RAM capacity is a major factor in determining laptop pricing.

### Q6: Average price of laptops with a spec score above 70
What is the average price of laptops with a spec score above 70?

In [13]:
%%sql
SELECT ROUND(AVG(price), 2) 
    AS avg_price_above_70_spec_score
FROM laptops
WHERE spec_score > 70;

1 rows affected.

avg_price_above_70_spec_score
182792.22


### Q7: Top 5 laptop brands with highest average user rating
Which are laptop brands have the top 5 highest average user rating?

In [14]:
%%sql
SELECT brand, 
    ROUND(AVG(user_rating), 1) 
        AS avg_user_ratings
FROM laptops
GROUP BY brand
ORDER BY avg_user_ratings DESC
LIMIT 5;

5 rows affected.

brand,avg_user_ratings
Colorful,4.6
Huawei,4.5
LG,4.5
Microsoft,4.4
Tecno,4.4


### Q8: Average warranty period by brand
What is the average warranty period offered by each brand?

In [15]:
%%sql
SELECT brand, 
       ROUND(AVG(
           CAST(SUBSTRING(warranty FROM '[0-9]+') 
                AS INTEGER)), 2) 
            AS avg_warranty_years
FROM laptops
GROUP BY brand
ORDER BY avg_warranty_years DESC;

31 rows affected.

brand,avg_warranty_years
Gigabyte,2.00
Fujitsu,2.00
MSI,1.80
Xiaomi,1.17
Lenovo,1.11
Huawei,1.00
Ultimus,1.00
Ninkear,1.00
Walker,1.00
Jio,1.00


### Q9: Brand market share across different price segments
How does the market share of different brands vary across different price segments (budget, mid-range, premium)?

In [17]:
%%sql

-- first, create price segments for the laptops
-- Budget: at least 40,000 , 
-- Mid-range: 40,001 - 100,000, 
-- Premium: above 100,000

-- I will create a view to have a consistent price segment 
-- for all questions

-- create view of the laptop price segment

CREATE VIEW laptop_price_segments AS
SELECT id, brand, name, price,
    CASE WHEN price <= 40000 THEN 'Budget'
         WHEN price BETWEEN 40001 AND 100000 THEN 'Mid-range'
         ELSE 'Premium' END AS price_segment
FROM laptops;

In [18]:
%%sql

-- next, find the market share, which is:
-- (brand laptop count / total laptop count in price segment) * 100

WITH brand_cte AS (
SELECT brand, 
    price_segment, 
    COUNT(*) AS laptop_count
FROM laptop_price_segments
GROUP BY brand, price_segment
)
SELECT price_segment, brand, 
    laptop_count, 
    SUM(laptop_count) OVER(PARTITION BY price_segment) 
    AS total_in_segment, 
    ROUND(100.0 * laptop_count / 
          SUM(laptop_count) OVER(PARTITION BY price_segment), 2) 
    AS market_share
FROM brand_cte
ORDER BY price_segment, market_share DESC;

52 rows affected.

price_segment,brand,laptop_count,total_in_segment,market_share
Budget,Lenovo,42,201,20.90
Budget,HP,34,201,16.92
Budget,Asus,33,201,16.42
Budget,Acer,27,201,13.43
Budget,Dell,16,201,7.96
Budget,Infinix,13,201,6.47
Budget,Chuwi,8,201,3.98
Budget,Zebronics,6,201,2.99
Budget,MSI,4,201,1.99
Budget,Ultimus,4,201,1.99


**1. Market Dominance:** 

In the `budget segment`, **Lenovo leads with 20.90% market share**, followed closely by HP (16.92%) and Asus (16.42%). 

The `mid-range segment` is also **dominated by Lenovo (22.83%)**, Asus (20.35%), and HP (20.18%), showing their strong presence across both budget and mid-range markets. 

In the `premium segment`, **HP takes the lead with 20.26% market share**, closely followed by MSI at 19.83%, indicating HP's strong presence across all segments.

**2. Brand Diversity:**

The `budget segment` shows the **most brand diversity** with 18 different brands represented, including some **lesser-known names** like **Chuwi, Ultimus, and Primebook**. 

The `mid-range segment` has 21 brands, but with a **more concentrated market share** among the top brands. 

The `premium segment` is the **least diverse** with only 13 brands and it is dominated by **well-established names** like **HP, MSI, Lenovo, Asus and Dell**.

**3. Segment Distribution:**

The `mid-range segment` is the largest with **565 laptops**, followed by the `premium segment` with **232 laptops**, and the `budget segment` with **235 laptops**. 

Some brands like Apple and Samsung have a stronger presence in the premium segment compared to budget and mid-range. 

Certain brands **(e.g., Infinix, Zebronics) appear in budget** and **mid-range** but *not in the premium segment,* indicating their market focus.

### Q10: Average user rating in different utility categories
What is the average user rating for laptops in different utility categories (e.g., gaming, business, everyday use) across brands?

In [19]:
%%sql
SELECT utility, 
    ROUND(AVG(user_rating), 2) 
        AS avg_user_rating
FROM laptops
WHERE utility IS NOT NULL
GROUP BY utility
ORDER BY avg_user_rating DESC;

22 rows affected.

utility,avg_user_rating
"Business, Everyday Use, Performance",4.60
"Performance, Business",4.54
"Gaming, Everyday Use, Performance",4.50
"Everyday Use, Gaming, Business, Performance",4.48
"Everyday Use, Gaming, Performance",4.45
"Everyday Use, Performance, Gaming",4.45
"Everyday Use, Business",4.42
"Everyday Use, Business, Performance, Gaming",4.38
"Performance, Everyday Use",4.37
"Gaming, Performance",4.36


The output shows that laptops with multiple utilities tend to have **higher average user rating** with the *highest rating (4.60)* going to the `"Business, Everyday Use, Performance"` utility category.

The *second highest (4.54)* goes to `"Performance, Business"` category.

Interesting, laptops categorized **soley for `gaming` or `business`** have *lower average ratings (4.36 and 4.27 respectively)* compared to multi-purpose laptops. This may suggest that users tend to rate versatile laptops more favorably than specialized ones.

### Q11: Comparison of Gaming Laptop Brands 
Among the laptops specifically marketed as gaming devices, which brand offers the highest number, and how does this compare to other gaming-focused brands in the dataset?

In [35]:
%%sql

-- I will only consider laptops listed as 'Gaming Laptop' in their name,
-- or solely for Gaming in the utility category

WITH gaming_brands_cte AS
(
SELECT brand, 
    COUNT(*) AS num_of_gaming_laptop, 
    ROUND(AVG(price), 2) AS avg_price, 
    ROUND(AVG(spec_score), 2) AS avg_spec_score
FROM laptops
WHERE name LIKE '%Gaming%' OR utility = 'Gaming'
GROUP BY brand
)
SELECT brand, 
    num_of_gaming_laptop, 
    CONCAT(ROUND(100.0 * num_of_gaming_laptop / 
        (SELECT SUM(num_of_gaming_laptop) FROM gaming_brands_cte), 2), 
           '%') 
    AS gaming_market_share,
    avg_price,
    avg_spec_score
FROM gaming_brands_cte
ORDER BY num_of_gaming_laptop DESC;

11 rows affected.

brand,num_of_gaming_laptop,gaming_market_share,avg_price,avg_spec_score
MSI,79,23.58%,130375.27,72.80
HP,66,19.70%,93888.97,68.62
Asus,58,17.31%,99108.45,69.31
Lenovo,56,16.72%,114911.86,68.57
Acer,37,11.04%,79840.95,61.62
Dell,30,8.96%,181365.87,74.27
Gigabyte,5,1.49%,102392.00,69.80
Xiaomi,1,0.30%,102990.00,78.00
Razer,1,0.30%,199990.00,83.00
Ninkear,1,0.30%,79999.00,73.00


**MSI dominates the gaming laptop market** with a **23.58% share (79 models)**, followed by **HP (19.70%, 66 models)** and **Asus (17.31%, 58 models)**. This shows a clear lead by these three manufactures in terms of product variety.

Other notable brands include **Lenovo with 56 gaming laptops** and **market share of 16.72%**. Also, **Dell** have fewer gaming laptops of **30 models** but boasts the **highest average price (181,365.87)** and **second-highest average spec score (74.27)**.

Brands like **Gigabyte, Xiaomi, Razer, Ninkear, and Colorful** have very few offerings, each having just *one to five gaming laptops*.

### Q12: Price-Rating Discrepancy Within Generations
Which laptops have a higher than average price but lower than average user ratings within their processor generation?

Output the top 3 laptops in each generation with the biggest price discrepancies.

In [21]:
%%sql

WITH gen_cte AS
(
SELECT processor_gen, 
    ROUND(AVG(price), 2) AS gen_avg_price, 
    ROUND(AVG(user_rating), 2) AS gen_avg_user_rating
FROM laptops
WHERE processor_gen IS NOT NULL
GROUP BY processor_gen
)
, ranked_laptops AS 
(
SELECT l.processor_gen, l.name, l.price, l.user_rating, 
    g.gen_avg_price, g.gen_avg_user_rating,
    (l.price - g.gen_avg_price) AS price_diff,
    (g.gen_avg_user_rating - l.user_rating) AS rating_diff,
    ROW_NUMBER() OVER(PARTITION BY l.processor_gen 
    ORDER BY (l.price - g.gen_avg_price) 
            - (l.user_rating - g.gen_avg_user_rating) DESC) 
    AS row_num
FROM laptops l
    JOIN gen_cte g 
    ON l.processor_gen = g.processor_gen
    WHERE l.price > g.gen_avg_price 
    AND l.user_rating < g.gen_avg_user_rating
)
SELECT 
    processor_gen, name,
    price, user_rating, 
    gen_avg_price, gen_avg_user_rating,
    price_diff, rating_diff
FROM ranked_laptops
WHERE row_num <= 3
ORDER BY processor_gen, row_num;

29 rows affected.

processor_gen,name,price,user_rating,gen_avg_price,gen_avg_user_rating,price_diff,rating_diff
10th Gen,Dell Precision 5550 Laptop,184999,4.2,57431.00,4.23,127568.00,0.03
11th Gen,Wings Nuvobook Pro Laptop,59990,4.1,46468.53,4.26,13521.47,0.16
11th Gen,MSI GF63 Thin 11UC-1295IN Gaming Laptop,58990,4.2,46468.53,4.26,12521.47,0.06
11th Gen,Lenovo IdeaPad 3 82H803HRIN Laptop,56990,4.2,46468.53,4.26,10521.47,0.06
12th Gen,HP ZBook Studio G9 2023 Workstation WQUXGA Laptop,390914,4.3,67900.91,4.32,323013.09,0.02
12th Gen,HP ZBook Studio G9 WUXGA 16 Workstation Laptop,373192,4.2,67900.91,4.32,305291.09,0.12
12th Gen,Lenovo ThinkPad P14s 21AKS02800 Laptop,133990,4.3,67900.91,4.32,66089.09,0.02
13th Gen,MSI CreatorPro X17 HX A13VKS-249IN Laptop,519990,4.1,114701.20,4.34,405288.80,0.24
13th Gen,Dell Alienware X16 Gaming Laptop,453990,4.3,114701.20,4.34,339288.80,0.04
13th Gen,Lenovo Legion 9i 83AG0044IN Laptop,449990,4.3,114701.20,4.34,335288.80,0.04


The analysis reveals laptops from various processor generations (3rd Gen to 14th Gen, including M3 series for Apple brand) that exhibit higher-than-average prices but lower-than-average user ratings within their respective generations. 

We can see the **most noticeable discrepancy** in the `13th Gen` category, where the **MSI CreatorPro X17 HX A13VKS-249IN Laptop** is priced **405,288.80** above the generation average but having a `user rating` **0.24 points** below average. 

Among Intel generations, the `14th Gen` category has the **highest average prices**, with *all three listed models exceeding 300,000 above the generation average.*

For `Apple M3 series models`, the MacBook Pro consistently show above-average prices with slightly lower ratings. This indicates a trend where even newer models may not meet user expectations despite their premium pricing.

### Q13: Top-Tier Utility Laptops
Identify the most expensive laptop in each utility category (e.g., Gaming, Business, Everyday Use).

In [22]:
%%sql
SELECT utility, 
    name AS most_expensive_laptop, 
    price
FROM
(
SELECT utility, name, price, 
    RANK() OVER(PARTITION BY utility ORDER BY price DESC) 
    AS price_rank
FROM laptops
WHERE utility IS NOT NULL
) AS ranked_laptops
WHERE price_rank = 1
ORDER BY price DESC;

22 rows affected.

utility,most_expensive_laptop,price
"Gaming, Performance",MSI Titan 18 HX Gaming Laptop,599990
Performance,MSI CreatorPro X17 HX A13VKS-249IN Laptop,519990
Gaming,Dell Alienware X16 Gaming Laptop,453990
Everyday Use,HP Spectre 17-cs0097nr Foldable Laptop,415000
"Business, Performance",HP ZBook Studio G9 2023 Workstation WQUXGA Laptop,390914
Business,HP Dragonfly G4 Laptop,231746
"Performance, Gaming",MSI Stealth 14 AI Studio A1VGG-054IN Gaming Laptop,229990
"Performance, Business, Everyday Use",Lenovo Yoga Book 9i 2023 82YQ001DIN Laptop,222990
"Everyday Use, Gaming",HP Omen 16-wf0056TX 834U0PA Gaming Laptop,153490
"Performance, Business",LG gram360 16 Laptop,152499


The data shows the most expensive laptops across different utility categories. `Gaming and performance` laptops command the **highest price** in these categories, for example, the **MSI Titan 18 HX Gaming Laptop** which is priced at **599,990** in the `"Gaming, Performance"` category.

The **prices vary significantly across categories**, from high-end specialized laptops like the HP ZBook Studio G9 for "Business, Performance" at 390,914, to more versatile options like the Lenovo V15 G3 Laptop for "Everyday Use, Business, Performance, Gaming" at 45,900. 

This indicates that laptops designed for multiple utilities tend to be more moderately prices that those optimized for high-performance tasks.

### Q14: Elite Performance Laptops: Triple Top 100 Analysis
Find laptops that simultaneously appear in the top 100 rankings for price, spec score, and user rating.

In [23]:
%%sql
WITH most_expensive_laptops AS 
(
SELECT brand, name, price, spec_score, user_rating
FROM laptops
ORDER BY price DESC
LIMIT 100
), high_performance_laptops AS 
(
SELECT brand, name, price, 
    spec_score, user_rating
FROM laptops
ORDER BY spec_score DESC
LIMIT 100
), most_rated_laptops AS
(
SELECT brand, name, 
    price, spec_score, user_rating
FROM laptops
ORDER BY user_rating DESC
LIMIT 100
)
SELECT * 
FROM most_expensive_laptops
INTERSECT
SELECT *
FROM high_performance_laptops
INTERSECT
SELECT *
FROM most_rated_laptops
ORDER BY spec_score DESC;

12 rows affected.

brand,name,price,spec_score,user_rating
MSI,MSI Stealth 17 A13VH-055IN Gaming Laptop,489883,98,4.7
Dell,Dell Alienware m16 Gaming Laptop,303290,91,4.7
Dell,Dell XPS 17 9730 Laptop,303490,87,4.7
Dell,Dell Alienware X16 R1 Gaming Laptop,386990,83,4.7
HP,HP Omen 16-u0023TX Gaming Laptop,237990,82,4.7
MSI,MSI Prestige 16 AI Studio B1VEG Laptop,159990,80,4.8
Dell,Dell Alienware m16 Gaming Laptop,220990,80,4.7
HP,HP Omen 16-wf1026TX Gaming Laptop,176499,80,4.7
Lenovo,Lenovo Thinkpad P15 20YRS3A300 Laptop,159999,79,4.7
HP,HP Omen 14-fb0007TX Gaming Laptop,191668,77,4.7


The results reveals seven high-end elite laptops that simultaneously appear in the top 100 rankings for price, spec score and user rating. 

Just four brands are in the output **(MSI, Dell, HP, and Lenovo)**, with the **MSI Stealth 17 A13VH-055IN Gaming Laptop** leading the pack with the highest price (489,883), spec score (98), and having a user rating of 4.7.

**Dell and HP** each have **multiple entries** in this elite group, with Dell's offerings (Alienware m16 and XPS 17 9730) commanding high prices and impressive spec scores, while HP's Omen series demonstrates a strong presence across various price points.

All seven laptops share user ratings of 4.7 and above, which indicates consistent high user satisfaction across these elite models, despite their price and spec score variations.

### Q15: Best Value High-Performance Laptops by Processor Brand
Identify the top 5 laptops with the best value (lowest price-to-spec score ratio) or Intel and AMD processors separately, considering only laptops with a spec score above 70.

In [24]:
%%sql
WITH value_laptops AS (
SELECT brand, name, 
    price, spec_score,
CASE WHEN processor_brand ILIKE '%Intel%' 
    THEN 'Intel'
     WHEN processor_brand ILIKE '%AMD%' 
    THEN 'AMD'
     ELSE 'Other' 
    END AS processor_type,
    ROUND(price::DECIMAL / spec_score, 2) 
        AS price_to_spec_ratio,
    ROW_NUMBER() OVER (
        PARTITION BY 
    CASE WHEN processor_brand ILIKE '%Intel%' 
        THEN 'Intel'
         WHEN processor_brand ILIKE '%AMD%' 
        THEN 'AMD'
             ELSE 'Other' 
        END ORDER BY (price::DECIMAL / spec_score) ASC) 
    AS rank
FROM laptops 
    WHERE spec_score > 70
)
SELECT 
    processor_type, brand, name, 
    price, spec_score, price_to_spec_ratio
FROM value_laptops
WHERE processor_type IN ('Intel', 'AMD') 
AND rank <= 5
ORDER BY processor_type, price_to_spec_ratio;

10 rows affected.

processor_type,brand,name,price,spec_score,price_to_spec_ratio
AMD,Ninkear,Ninkear A16 Gaming Laptop,79999,73,1095.88
AMD,HP,HP Victus 16-s0095AX Gaming Laptop,79990,72,1110.97
AMD,HP,HP Victus 16-s0094AX Gaming Laptop,81490,73,1116.30
AMD,HP,HP Victus 16-s0089AX Gaming Laptop,80490,72,1117.92
AMD,Asus,Asus ROG Flow X13 GV301RC-LJ132WS Gaming Laptop,84990,74,1148.51
Intel,HP,HP Victus 16-D0314TX Gaming Laptop,60200,71,847.89
Intel,MSI,MSI Thin GF63 11UCX-1491IN Gaming Laptop,61990,71,873.10
Intel,Asus,Asus TUF Gaming F15 2022 FX507ZC4-HN116W Gaming Laptop,72990,71,1028.03
Intel,Asus,Asus TUF Gaming F15 FX506HE-HN385WS Gaming Laptop,74990,71,1056.20
Intel,Asus,Asus TUF Gaming F15 FX577ZC-HN192W Gaming Laptop,84990,72,1180.42


The output reveals that `Intel` laptops generally offer a **better price-to-spec score ratio** compared to `AMD` laptops, suggesting **better value for money** in high performance models.

Among AMD laptops, the "Ninkear A16 Gaming Laptop" offers the best value, with a price-to-spec score ratio of 1095.88, while HP dominates this top 5 list with multiple models. 

For Intel laptops, the "HP Victus 16-D0314TX Gaming Laptop" stands out with the lowest price-to-spec score ratio of 847.89.

Also, we see that **HP and Asus** are prominent brands in providing value in both Intel and AMD high-performance gaming laptops.

### Q16: Percentage of Laptops with Touch Screen
What percentage of laptops offer touch screen functionality, and how does this correlate with price?

In [33]:
%%sql

-- showing percentage of laptops with touch screen feature
-- in the entire dataset

SELECT 
    CONCAT(
        ROUND(100.0 * 
    SUM(CASE WHEN screen_feature LIKE '%Touch Screen%' 
        THEN 1 ELSE 0 END) 
     / COUNT(*), 2), '%') 
    AS pct_of_laptops_with_touch_screen
FROM laptops;

1 rows affected.

pct_of_laptops_with_touch_screen
10.32%


In [28]:
%%sql

-- showing how laptops offering touch screen functionality
-- correlate with price

WITH touch_screen_cte AS
(
SELECT 
    lps.price_segment AS price_segment,
    ROUND(AVG(l.price), 2) AS avg_price_all,
    ROUND(AVG(CASE WHEN l.screen_feature LIKE '%Touch Screen%' 
              THEN l.price END), 2) AS avg_price_touch_screen,
    SUM(CASE WHEN l.screen_feature LIKE '%Touch Screen%' 
        THEN 1 ELSE 0 END) AS touch_screen_laptop_count,
    COUNT(*) AS total_laptops
FROM laptops l
JOIN laptop_price_segments lps -- from laptop_price_segments view
ON l.id = lps.id
GROUP BY lps.price_segment
)
SELECT *, 
    ROUND(100.0 * touch_screen_laptop_count 
          / total_laptops, 2) 
    AS perc_of_touch_screen_laptop
FROM touch_screen_cte
ORDER BY price_segment;

3 rows affected.

price_segment,avg_price_all,avg_price_touch_screen,touch_screen_laptop_count,total_laptops,perc_of_touch_screen_laptop
Budget,30571.36,26240.00,4,201,1.99
Mid-range,65447.31,78608.38,56,565,9.91
Premium,178995.73,176134.26,43,232,18.53


The data reveals a **clear positive correlation** between `price segment` and the `touch screen functionality` in laptops.

`Budget laptops` (average price 30,571) have the **lowest touch screen adoption rate** at just 1.99% of 201 models.

The percentage of laptops with touch screens **increases** significantly in **higher price segments**, with 9.91% of 565 `mid-range laptops` (average price 65,447). The `premium segment` have the **highest number of touch screen laptops** with 18.53% of 232 models (average price 178,996) having this feature. 

This indicates that touch screens are increasingly becoming a standard feature in higher-end laptops.

Also, we find out that touch screen laptops are **generally more expensive** in the `Mid-range segment`, but slightly cheaper in the Premium segment compared to the average price of all laptops in that segment.

### Q17: Biometric Security Adoption Analysis 
Analyze the adoption rate of fingerprint sensors across different price ranges.

In [32]:
%%sql

-- showing percentage of laptops with fingerprint sensors
-- in the entire dataset

SELECT 
CONCAT(ROUND(100.0 * 
SUM(CASE WHEN hardware_features LIKE '%Fingerprint Sensor%' 
        THEN 1 ELSE 0 END) 
     / COUNT(*), 2), '%') AS pct_of_laptops_with_fingerprint_sensor
FROM laptops;

1 rows affected.

pct_of_laptops_with_fingerprint_sensor
27.66%


In [31]:
%%sql

-- showing percentage of laptops with fingerprint sensor 
-- by price segment

WITH fingerprint_sensor_cte AS
(
SELECT lps.price_segment AS price_segment,
    ROUND(AVG(l.price), 2) AS avg_price_all,
    ROUND(AVG(CASE WHEN l.hardware_features LIKE '%Fingerprint Sensor%' 
              THEN l.price END), 2) AS avg_price_fingerprint_sensor,
    SUM(CASE WHEN L.hardware_features LIKE '%Fingerprint Sensor%' 
        THEN 1 ELSE 0 END) AS fingerprint_laptop_count,
    COUNT(*) AS total_laptops
FROM laptops l
JOIN laptop_price_segments lps -- from laptop_price_segments view
ON l.id = lps.id  
GROUP BY lps.price_segment
)
SELECT *, 
ROUND(100.0 * fingerprint_laptop_count / 
      total_laptops , 2) 
    AS perc_of_laptop_with_fingerprint_sensor
FROM 
fingerprint_sensor_cte;

3 rows affected.

price_segment,avg_price_all,avg_price_fingerprint_sensor,fingerprint_laptop_count,total_laptops,perc_of_laptop_with_fingerprint_sensor
Budget,30571.36,34349.44,18,201,8.96
Mid-range,65447.31,67338.86,184,565,32.57
Premium,178995.73,199544.97,74,232,31.90


The data shows a **clear correlation** between laptop `price segments` and the `adoption of fingerprint sensors`. 

`Budget laptops` (average price 30,571) having the **lowest adoption rate** at only 8.96% of 201 models.

Interestingly, `mid-range laptops` (average price 65,447) have the **highest adoption rate of fingerprint sensors** at 32.57% of 565 models, which is slightly higher than premium laptops (average price 178,996) at 31.90% of 232 models.

This may suggest that fingerprint sensors have become a standard feature in mid-range and premium laptops but are still considered a luxury in budget models.

As seen by the output, in all the 3 segments, **laptops with fingerprint sensors are more expensive** than the average price of all laptops in that segment.